In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import PIL
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.utils import load_img
from tensorflow.keras import layers
import time

import glob
import imageio

from IPython import display

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
X = np.load('/content/drive/MyDrive/X_line_class .npy')
Y = np.load('/content/drive/MyDrive/Y_gender_line_class.npy')

In [4]:
# def plot_random_samples(x, y):
#   plt.figure()

#   f, axarr = plt.subplots(2, 2, figsize=(10,10)) 
#   rands = np.random.randint(x.shape[0], size=4)
#   axarr[0, 0].imshow((x[rands[0], :, :,:]))
#   axarr[0, 0].set_title('Label: ' + str(y[rands[0]]), fontweight='bold')

#   axarr[1, 0].imshow((x[rands[1], :, :,:]))
#   axarr[1, 0].set_title('Label: ' + str(y[rands[1]]), fontweight='bold')

#   axarr[0, 1].imshow((x[rands[2], :, :,:]))
#   axarr[0, 1].set_title('Label: ' + str(y[rands[2]]), fontweight='bold')

#   axarr[1, 1].imshow((x[rands[3], :, :,:]))
#   axarr[1, 1].set_title('Label: ' + str(y[rands[3]]), fontweight='bold')

#   plt.show()

In [5]:
# plot_random_samples(X, Y)

In [6]:
# plt.imshow(X[0])

In [7]:
# X.shape

#PixelCNN

In [8]:
X_train_images = X / 255.0  # Normalize the images to [0, 1]
# plot_random_samples(X_train_images, Y)

In [9]:
batch_size = 128
epochs = 20
learning_rate = 0.001
num_classes = 2
input_shape = (75, 256, 3)

In [18]:
def PixelCNN(input_shape, num_classes):
  input_layer = tf.keras.layers.Input(shape=input_shape)

  # First layer
  conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(7, 7), padding='same', activation='relu')(input_layer)

  # Residual blocks
  for _ in range(15):
      residual = conv1
      conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(conv1)
      conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(1, 1), padding='same', activation='relu')(conv1)
      conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(1, 1), padding='same')(conv1)
      conv1 = tf.keras.layers.Add()([residual, conv1])
      conv1 = tf.keras.layers.Activation('relu')(conv1)

  # Final layer
  conv1 = tf.keras.layers.Conv2D(filters=256, kernel_size=(1, 1), padding='same', activation='relu')(conv1)
  output_layer = tf.keras.layers.Conv2D(filters=num_classes, kernel_size=(1, 1), padding='same', activation='softmax')(conv1)

  # Model
  model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

  return model

In [19]:
model = PixelCNN(input_shape, num_classes)
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate), metrics=['accuracy'])

In [20]:
model.fit(X_train_images[:100], X_train_images[:100], batch_size=batch_size, epochs=epochs)

Epoch 1/20


ValueError: ignored

In [ ]:
model.save('pixelcnn_model.h5')

In [ ]:
# Load model
model = tf.keras.models.load_model('pixelcnn_model.h5')

# Define class
class_id = 1  # Change this to the desired class ID

# Generate image
generated_image = np.zeros((75, 256, 3), dtype=np.uint8)
for i in range(75):
    for j in range(256):
        logits = model.predict(np.expand_dims(generated_image, axis=0))
        probs = tf.nn.softmax(logits)[0, i, j, :]
        generated_image[i, j, :] = tf.random.categorical(tf.math.log(probs), num_samples=1)[0, 0]

# Save image
# filename = f'class_{class_id}_generated_image.png'
# tf.keras.preprocessing.image.save_img(filename, generated_image)

plt.imshow(generated_image)